In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Convert between Vertex AI Vizier and Open Source Vizier

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/vizier/gapic-vizier-multi-objective-optimization.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/vizier/gapic-vizier-multi-objective-optimization.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/vizier/gapic-vizier-multi-objective-optimization.ipynb">
        <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>

## Overview

This tutorial demonstrates [Vertex AI Vizier](https://cloud.google.com/vertex-ai/docs/vizier/overview) and how to convert the code to [Open Source(OSS) Vizier](https://oss-vizier.readthedocs.io/). OSS Vizier is a Python-based service for blackbox optimization and research. It allows a user to setup an OSS Vizier Server, which can host blackbox optimization algorithms to tune the objective functions, and to define abstractions and utilities for implementing new optimization algorithms. 

If you are an existing OSS Vizier user, the instructions show you how to migrate the code to Vertex Vizier SDK. If you are a user of Vertex Vizier, the instructions show you how to modify the code to adapt to OSS Vizier.

### Objective

In this tutorial, you learn how to use `Vertex AI Vizier` to optimize a multi-objective study and convert the code to OSS Vizier.

The goal is to __`minimize`__ the objective metric:
   ```
   y1 = r*sin(theta)
   ```

and simultaneously __`maximize`__ the objective metric:
   ```
   y2 = r*cos(theta)
   ```

that you will evaluate over the parameter space:

   - __`r`__ in [0,1],

   - __`theta`__ in [0, pi/2]

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing) and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.


## Installation

Install the packages required for executing this notebook.

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME") and not os.getenv("VIRTUAL_ENV")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install google-vizier==0.0.4
! pip3 install --upgrade google-cloud-aiplatform {USER_FLAG} -q

### Restart the kernel

Once you've installed the additional packages, you need to restart the notebook kernel so it can find the packages.

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Restart the kernel after pip installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Before you begin

### GPU runtime

This tutorial does not require a GPU runtime.

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project.](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [Enable the Vertex AI APIs, Compute Engine APIs, and Cloud Storage.](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component,storage-component.googleapis.com)

4. [The Google Cloud SDK](https://cloud.google.com/sdk) is already installed in Google Cloud Notebook.

5. Enter your project ID in the cell below. Then run the  cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$`.

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your Google Cloud project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for Vertex AI. We recommend that you choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You may not use a multi-regional bucket for training with Vertex AI. Not all regions provide support for all Vertex AI services.

Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations)

In [ ]:
REGION = "[your-region]"  # @param {type:"string"}

In [ ]:
if REGION == "[your-region]":
    REGION = "us-central1"

### Authenticate your Google Cloud account

**If you are using Vertex AI Workbench Notebook**, your environment is already authenticated. Skip this step.

**If you are using Colab**, run the cell below and follow the instructions
when prompted to authenticate your account via oAuth.

**Otherwise**, follow these steps:

1. In the Cloud Console, go to the [**Create service account key**
   page](https://console.cloud.google.com/apis/credentials/serviceaccountkey).

2. Click **Create service account**.

3. In the **Service account name** field, enter a name, and
   click **Create**.

4. In the **Grant this service account access to project** section, click the **Role** drop-down list. Type "Vertex AI"
into the filter box, and select
   **Vertex AI Administrator**. Type "Storage Object Admin" into the filter box, and select **Storage Object Admin**.

5. Click *Create*. A JSON file that contains your key downloads to your
local environment.

6. Enter the path to your service account key as the
`GOOGLE_APPLICATION_CREDENTIALS` variable in the cell below and run the cell.

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your Google Cloud account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your Google Cloud
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### Import libraries and define constants

In [ ]:
import datetime
import math

## Tutorial


This section defines some parameters to create the study and optimize the objective function.


In [ ]:
# These will be automatically filled in.
STUDY_DISPLAY_NAME = "{}_study_{}".format(
    PROJECT_ID.replace("-", ""), datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
)

print("REGION: {}".format(REGION))

### Define the parameters

The following is a sample study configuration, built as a hierarchical python dictionary. It is already filled out. Run the cell to configure the study.

__`USE_VERTEX_VIZIER`__: Uses Vertex Vizier SDK to do the optimization if True. Use OSS Vizier otherwise.

__`SUGGESTION_COUNT`__: The number of suggestions (trials) requested in a single request.

__`MAX_NUM_ITERATIONS`__: The number of iterations to explore before stopping. It is set to 4 to shorten the time to run the code, so don't expect convergence. For convergence, it would likely need to be about 20.


In [ ]:
USE_VERTEX_VIZIER = True  # @param {type:"boolean"}

MAX_NUM_ITERATIONS = 4  # @param {type:"integer"}

SUGGESTION_COUNT = 2  # @param {type:"integer"}

OWNER = "owner"  # @param {type:"string"}

SERVICE_ENDPOINT = "127.0.0.1:8888"  # @param {type:"string"}

### Import the package and define CreateStudy for different sources

In Vertex Vizier, `project` and `location` are specified and `Study.create_or_load` is called to create a study. In OSS Vizier, you'll need to input the owner of your study and the server address with format [ip:port]. To bring up the OSS Vizier server, please follow the [instruction](https://oss-vizier.readthedocs.io/) on the OSS Vizier website.

In [ ]:
if USE_VERTEX_VIZIER:
    from google.cloud import aiplatform
    from google.cloud.aiplatform.vizier import Study, pyvizier

    def CreateStudy(project, location, display_name, problem):
        aiplatform.init(project=project, location=location)
        study = Study.create_or_load(display_name=display_name, problem=problem)
        return study

else:
    from vizier.service import clients, pyvizier

    def CreateStudy(project, location, display_name, problem):
        clients.environment_variables.service_endpoint = SERVICE_ENDPOINT
        study = clients.Study.from_study_config(
            problem, owner=OWNER, study_id=STUDY_DISPLAY_NAME
        )
        return study

### Metric evaluation functions

Next, define some functions to evaluate the two objective metrics.


In [ ]:
# r * sin(theta)
def Metric1Evaluation(r, theta):
    """Evaluate the first metric on the trial."""
    return r * math.sin(theta)


# r * cos(theta)
def Metric2Evaluation(r, theta):
    """Evaluate the second metric on the trial."""
    return r * math.cos(theta)


def CreateMetrics(r, theta):
    # Evaluate both objective metrics for this trial
    y1 = Metric1Evaluation(r, theta)
    y2 = Metric2Evaluation(r, theta)
    print(
        "[r = {}, theta = {}] => y1 = r*sin(theta) = {}, y2 = r*cos(theta) = {}".format(
            r, theta, y1, y2
        )
    )
    measurement = pyvizier.Measurement()
    measurement.metrics["y1"] = y1
    measurement.metrics["y2"] = y2

    # Return the results for this trial
    return measurement

### Optimization

The following code defines a study with parameters and metrics, evaluates the metric information based on the suggestions from Vizier, and reports the metrics value back. After a few rounds of iteration, you can get optimal trials by calling `optimal_trials()`. The code is adapt to both Vertex Vizier and OSS Vizier.

In [ ]:
problem = pyvizier.StudyConfig()
problem.algorithm = pyvizier.Algorithm.RANDOM_SEARCH

# Objective Metrics
problem.metric_information.append(
    pyvizier.MetricInformation(name="y1", goal=pyvizier.ObjectiveMetricGoal.MINIMIZE)
)
problem.metric_information.append(
    pyvizier.MetricInformation(name="y2", goal=pyvizier.ObjectiveMetricGoal.MAXIMIZE)
)

# Defines the parameters configuration.
root = problem.search_space.select_root()
root.add_float_param("r", 0, 1.0, scale_type=pyvizier.ScaleType.LINEAR)
root.add_float_param("theta", 0, 1.57, scale_type=pyvizier.ScaleType.LINEAR)

study = CreateStudy(
    project=PROJECT_ID,
    location=REGION,
    display_name=STUDY_DISPLAY_NAME,
    problem=problem,
)

for _ in range(MAX_NUM_ITERATIONS):
    trials = study.suggest(count=SUGGESTION_COUNT)
    for trial in trials:
        materialize_trial = trial.materialize()
        measurement = CreateMetrics(
            materialize_trial.parameters.get_value("r"),
            materialize_trial.parameters.get_value("theta"),
        )
        trial.add_measurement(measurement=measurement)
        trial.complete(measurement=measurement)

optimal_trials = study.optimal_trials()
print("optimal_trials: {}".format(optimal_trials))

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial. You can also manually delete resources that you created by running the following code.

In [ ]:
study.delete()